# SQLITE3  in Python

SQLlite is a package to manipulate dataset using the Structured Query Language.

In this notebook we introduce the concept of a SQL database and show how to interact with these databases in Python.

The fundamental concepts in SQL are the following:
* a database is a collection of datasets stored as tables.
  * Tables are similar to a pandas dataframe but the index is called rowid. 
  * We use SQL commands to create and drop tables in the database
  * CREATE allows us to create a new table
  * DROP allows us to delete an existing table and all of its data
* a table is a dataset consisting of a number of rows and columns
  *  each column has a name and a type
  *  each row has a unique rowid automatically created by SQLite
* a cursor is a Python object that allows us to interact with the tables in a database in several ways
  * INSERT allows us to add new rows
  * UPDATE allows us to modify the content of a row
  * DELETE allows us to remove rows
  * SELECT allows us to return rows of data from the tables in the database

Here is a link to all of the [SQLite statements](https://www.sqlite.org/lang.html) (and their grammar)

Here is a page 
[Pandas vs SQL](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html)
that compares Pandas with SQL commands since they both deal with tables of data and do similar things.

As usual we first import the libraries we will need.


In [ ]:
import sqlite3
import csv
'done'

# Interacting with the database
Sqlite stores its data in a file and to interact with the database you need to take three steps:
1. create a connection to the database (and this will create the database file if it doesn't already exist)
1. create a cursor for the connection (this allows us to query and modify the database)
1. run some queries using the cursor
1. commit the changes (if we don't commit the change then the database isn't updated, it is similar to commit with git!)
1. close the connection

The template for this code looks like this
``` python
    con= sqlite3.connect('courses.db') # create the connection
    cur = con.cursor()   # create the cursor to interact with the connection
    cur.execute(''' SOME 
    (MULTILINE) SQL COMMANDS GO HERE
    ending with a semicolon, always  ;
    ''')
    con.commit()  # write any database changes to the database file
    con.close()  # close the connection
```

In this tutorial we will learn a few of the core SQL commands. 
The full list is here https://www.sqlite.org/lang.html

# Overview of most common SQL commands
## Creating and dropping tables
``` python
CREATE TABLE tablename IF NOT EXISTS (colnam1 coltype1,  colname2 coltyp2, ...); -- create a new table
DROP TABLE tablename;
```
## Inserting data into a table
``` python
INSERT INTO tablename VALUES (col1val, col2val, ...) ; -- insert a row into a table
```
## Selecting data from a table
``` python
SELECT * FROM tablename;
SELECT colname1, colname2, ... FROM tablename WHERE condition1 OR condition2 AND NOT condition3;
```
## Delete rows from the table
``` python
DELETE FROM tablename;  -- remove all rows from the table
DELETE FROM tablename WHERE conditions;  -- delete selected rows from the table
```
## Updating data 
``` python
UPDATE tablename SET colname=expression WHERE condition;
```

# CRUD
There are many more SQL commands but these five will allow you to do all of the CRUD operations
which form the foundation of most database-backed apps. If you are interested in learning more about
databases you should take the **CS127b: Introduction to Databases** course
* Create -- with INSERT
* Read -- with SELECT
* Update -- with UPDATE
* Delete -- with DELETE

In the rest of this notebook we show how to interact with a SQLite database using Python.

We will use the courses.csv dataset as our example.

# Creating a table to store the course data
SQL requires that all data be stored in tables where each column has a name and a type.

The SQLite types are TEXT, NUMERIC, INTEGER, REAL, BLOB 

Here we define a function to create the table we are going to use to store the PA01 dataset.

The PA01 dataset, courses.csv, has the following form as a CSV file:
```
subj	num	suf	format	term	code	inst	title	sec	enr
NEJS	188	A	LEC	Fall 2004	1043	Levy, Avigdor	RISE/DECLINE:OTTOMAN EMP	1	32
CHEM	121	A	LEC	Fall 2004	1043	Foxman, Bruce	INORGANIC CHEM I, LECS.	1	18
CHEM	130	A	LEC	Fall 2004	1043	Yu, Jinquan	ADV ORG. CHEM: STRUCTURE	1	14
```
The functions below create or drop the table to store the courses.csv data.

In [ ]:
def create_data_table():
    ''' create a table to store the Brandeis course data'''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS data
                     (subj text, num int, suf text, format text, term text, code int, inst text, title text, sec text, enr int)''')
    con.commit()
    con.close()

def drop_data_table():
    ''' remove the table and all of its data from the database'''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    cur.execute('''DROP TABLE IF EXISTS data ''')
    con.commit()
    con.close()
    
drop_data_table()
create_data_table()

# loading and selecting the data
To load data into a database we can use the executemany cursor command
with an "INSERT SQL" command. Each row in rows is a tuple with 10 elements (matching the 10 question marks in the quer)




In [ ]:
def load_data():
    ''' load data from courses.csv into courses.db by inserting one row at a time '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    course_file = open('data/courses.csv')
    rows = csv.reader(course_file)
    for row in rows:
        cur.execute("INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",row)
    con.commit()
    con.close()

def load_data2():
    ''' load data from courses.csv into courses.db using the executemany cursor method '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    course_file = open('data/courses.csv')
    rows = csv.reader(course_file)
    cur.executemany("INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",rows)
    con.commit()
    con.close()
    
def get_all_data():
    ''' read all of the data and return it as a list of tuples '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    results = cur.execute("SELECT * FROM data")
    data= [x for x in results]
    con.commit()
    con.close()
    return data

def remove_all_data():
    ''' remove all of the data from the table '''
    con= sqlite3.connect('courses.db')
    cur = con.cursor()
    results = cur.execute("DELETE FROM data")
    con.commit()
    con.close()

load_data()


# Selecting the data
We can use the select statement to pick rows out of the database and then access those rows with
``` python
cur.fetchone() -- returns the next row in the result
cur.fetchman(N) -- returns the next N rows as a list
cur.fetchall() -- returns all the rows as a list
```

Here are some examples..

In [ ]:
con= sqlite3.connect('courses.db')
cur = con.cursor()


print("First 10 courses in the dataset")
cur.execute("SELECT * FROM data;")
for i in range(10):
    rows = cur.fetchmany(1)
    print(rows)
print('-'*40)


print("Last 10 courses in the dataset")
cur.execute("SELECT * FROM data;")
data = cur.fetchall();
for row in data[-10:]:
    print(row)
print('-'*40)


print("first 100 or fewer COSI courses taught in summer semesters")
cur.execute("SELECT * FROM data where subj='COSI' and code%10=2;")
data = cur.fetchmany(100)
for row in data:
    print(row)
print('-'*40)


con.commit()
con.close()


# Creating a SQLite shell

In [ ]:
def shell():
    
    
    command = input(">> ")
    while command !='quit':
        con= sqlite3.connect('courses.db')
        cur = con.cursor()
        
        cur.execute(command)
        results = cur.fetchall()
        for row in results[:5]:
            print(row)
        print('.'*40)
        for row in results[-5:]:
            print(row)
        con.commit()
        con.close()
        
        command = input(">> ")
    print('bye')
    
    

        
    

In [ ]:
shell()

In [ ]:
drop_data_table()
create_data_table()
data = get_all_data()
print(len(data))

In [ ]:

load_data()


In [ ]:
data = get_all_data()
len(data)

In [ ]:
print(data[1000])

In [ ]:
print('removing all data from the table')
remove_all_data()
data = get_all_data()
print('tbe table size is now',len(data))
print('.'*40)
print('loading all data into the table')
load_data()
data = get_all_data()
print('the table size is now',len(data))




In [ ]:
print('Calculating the unique term codes in the dataset')
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT distinct code from data ")
codes = [row[0] for row in cur.fetchall()]
codes = codes[1:]
print(codes)
con.close()

In [ ]:
print('calculating the number of students taking COSI courses each term (with double counting!)')
con= sqlite3.connect('courses.db')
cur = con.cursor()
#cur.execute("DELETE FROM data WHERE code NOTNULL")
cur.execute("SELECT code,sum(enr) as n ,max(enr) as m,sum(enr)/count(enr) as average from data  where enr>=8 group by code")
for row in cur.fetchall():
    print(row)
con.close()

In [ ]:
print('Printing out the bulletin of all COSI courses offered from Fall04 to Summer21')
con= sqlite3.connect('courses.db')
cur = con.cursor()
for code in codes[:2]:
    cur.execute("SELECT * from data where subj=(?) and code=(?) order by code, num",('COSI', code ))
    print("Year ",code)
    for row in cur.fetchall():
        print(row)
    print('------',end="\n\n")
con.close()

In [ ]:
print('printing out the COSI courses from Spring 2005')
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT num, suf, title, inst from data where subj=(?) and code=(?)",('COSI', 1051))
for row in cur.fetchall():
    print(row)
con.commit()
con.close()

In [ ]:
print("printing out the number of COSI courses taught by each COSI instructor from Fall04 to Summer21")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT inst,sum(enr) as n from data where subj=(?) group by inst order by n desc",('COSI', ))
for row in cur.fetchall():
    print(row)
con.commit()
con.close()

In [ ]:
print("printing out the enrollments in COSI courses from Fall2004 through Summer2021")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT term,sum(enr) as n from data where subj=(?) group by code order by code",('COSI', ))
for row in cur.fetchall():
    print(row)
    if row[0].startswith('Summer'):
        print()
con.commit()
con.close()

In [ ]:
print("printing out rowids which are unique keys for each row")
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT rowid,enr,*  from data where subj=(?)  order by enr desc limit 20",('COSI', ))
for row in cur.fetchall():
    print(row)
    if row[2].startswith('Summer'):
        print()
con.commit()
con.close()

In [ ]:
#print("%-25s %5s %-40s  %3s %5s %5s"%('instructor','id','','class','num','total','avg'))
con= sqlite3.connect('courses.db')
cur = con.cursor()
cur.execute("SELECT inst,title,count(*),sum(enr),group_concat(term),num,suf from data where subj=(?) group by inst,title order by inst asc ",('COSI',))
results = cur.fetchall()
for row in results:
    print("%-25s %4d%s %-40s  %3d %5d %5.1f\n   %s\n"%((row[0],row[5], row[6], row[1], row[2], row[3], row[3]/row[2], row[4])))
con.commit()
con.close()